# Introduction to Langchain

LangChain is a framework for developing applications powered by language models

- GitHub: https://github.com/langchain-ai/langchain
- Docs: https://python.langchain.com/docs/get_started

## Outlines
1. Main components -- Model, Prompt Template, Output Parser
2. Chains
3. Memory
4. Retriever (RAG)
5. Evaluation


## 0. Installation

In [1]:
%pip install langchain

## 1. Main Components

### 1.1. Model

#### Model - Local Llama

In [ ]:
%pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 59.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.20-cp310-cp310-manylinux_2_35_x86_64.whl size=1987633 sha256=c3ea7b09418d30bbe545e16384b502a46551afd36b6643e1c15e6219ad10c9a2
  Stored in directory: /root/.cache/pip/wheels/ef/f2/d2/0becb03047a348d7bd9a5b91ec88f4654d6fa7d67ea4e84d43
Successfully built llama-cpp-python


In [ ]:
!wget https://gpt4all.io/models/gguf/mistral-7b-instruct-v0.1.Q4_0.gguf

--2023-12-11 04:50:42--  https://gpt4all.io/models/gguf/mistral-7b-instruct-v0.1.Q4_0.gguf
Resolving gpt4all.io (gpt4all.io)... 104.26.0.159, 104.26.1.159, 172.67.71.169, ...
Connecting to gpt4all.io (gpt4all.io)|104.26.0.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4108916384 (3.8G)
Saving to: ‘mistral-7b-instruct-v0.1.Q4_0.gguf’

uct-v0.1.Q4_0.gguf   91%[=================>  ]   3.50G  70.1MB/s    eta 6s     

In [ ]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(
    model_path="/content/mistral-7b-instruct-v0.1.Q4_0.gguf",
    n_gpu_layers=1,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
llm("The first man on the moon was ... Let's think step by step")

'. We know that Neil Armstrong and Edwin "Buzz" Aldrin were the first two humans to walk on the moon as part of the Apollo 11 mission in July 1969. So, if we go back a generation earlier, to the Apollo 8 mission in December 1965, the first man in space was Frank R. Grissom. The first woman in space was Valentina Tereshkova in June 1963. And the first American woman in space was Sally Ride in July 1983. Now we\'re getting close! If we go back to the early days of human flight, the Wright brothers made their first successful powered flight in December 1903. But that wasn\'t exactly on the moon or in space. So let\'s go further back.\n\nThe first man-made object to reach Earth from another celestial body was a meteorite, which is essentially space debris left behind by an asteroid or comet. Meteorites have been falling to Earth for billions of years. \n\nSo, the answer is: The first man on the moon was not the first human in space'

#### Model - OpenAI

In [2]:
%pip install openai

In [3]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from google.colab import userdata

# Get it from https://platform.openai.com/account/api-keys
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

llm = OpenAI(temperature=0)
chat = ChatOpenAI()

In [4]:
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

In [5]:
llm(text)

'\n\nRainbow Socks Co.'

In [ ]:
chat(messages)

AIMessage(content='1. ChromaSock\n2. KaleidoSocks\n3. RainbowThreads\n4. ColorfulSteps\n5. VibrantFeet\n6. HueHosiery\n7. PopSock\n8. TechniHue\n9. ChromaticSoles\n10. SockSpectrum')

#### Model - Hugging Face (Free)

In [6]:
%pip install huggingface_hub

In [7]:
from langchain.llms import HuggingFaceHub
from google.colab import userdata

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

repo_id = "google/flan-t5-xxl"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options


llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [8]:
text = "What would be a good company name for a company that makes colorful socks?"
llm(text)

'Happy Socks'

#### Bonus: Using LLM as a question-answering bot

In [ ]:
text = """Question: What would be a good company name for a company that makes colorful socks?

Let's think step by step.

Answer: """

In [ ]:
llm(text)

'Socks are a basic product. Socks are colorful. Socks are a must have. Socks are a basic product. Socks are the foundation of a company. Socks are the foundation of a company that makes colorful socks. The answer: Socks.'

### 1.2. Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [ ]:
llm(prompt.format(product="colorful socks"))

'sock wranglers'

In [ ]:
template = """Question: {question}

Let's think step by step, and then summarize the final answer in this format:

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
prompt.format(question="What is a good name for a company that makes video games")

"Question: What is a good name for a company that makes video games\n\nLet's think step by step, and then summarize the final answer in this format:\n\nAnswer: "

In [ ]:
llm(prompt.format(question="What is a good name for a company that makes video games"))

'Video games are played on computers, and computers are called consoles. Consoles are made by companies called game console makers. A company would call itself a video game console maker. The answer: console maker.'

### 1.3. Output Parser

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [ ]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
prompt_template = PromptTemplate(template=review_template, input_variables=["text"],)

In [ ]:
llm(prompt_template.format(text=customer_review))

'gift, delivery_days, price_value = -1, -1'

#### Using output parsers

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [ ]:
review_template_with_instructions = """\
For the following text, extract the following information:

{format_instructions}

text: {text}
"""

In [ ]:
prompt_template_with_output = PromptTemplate(template=review_template_with_instructions, input_variables=["text"], partial_variables={"format_instructions": format_instructions})

In [ ]:
prompt_and_model = prompt_template_with_output | llm
output = prompt_and_model.invoke({"text": customer_review})

In [ ]:
result = output_parser.invoke(output)
print(result)

OutputParserException: ignored

In [ ]:
result["gift"]

'True'

### Bonus: Chaining Stuffs

Because all of the objects implements the `Runnable` interface. It can be chained together.

More info: https://python.langchain.com/docs/expression_language/why

In [ ]:
output_chain = prompt_template_with_output | llm | output_parser

In [ ]:
output_chain.invoke({"text": customer_review})

OutputParserException: ignored

## 2. Chains


### Old way

In [ ]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt_template_with_output, llm=llm, output_parser=output_parser)

llm_chain.run(text=customer_review)

OutputParserException: ignored

### New way

In [ ]:
output_chain = prompt_template_with_output | llm | output_parser
output_chain.invoke({"text": customer_review})

OutputParserException: ignored

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = (
 { "text": RunnablePassthrough() }
 | prompt_template_with_output
 | llm
 | output_parser
)
chain.invoke(customer_review)

OutputParserException: ignored

## 3. Memory

### Manipulating the memory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")


In [ ]:
memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?"}

In [ ]:
memory.save_context({"input": "how yo doin'"}, {"output": "fine. thank you!"})

In [ ]:
memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?\nHuman: how yo doin'\nAI: fine. thank you!"}

In [ ]:
template = """You are a nice chatbot having a conversation with a human.

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)

#### Without memory

In [ ]:
no_memory_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
)

In [ ]:
no_memory_chain({"question": "Hello, My name is Junior"})

{'question': 'Hello, My name is Junior',
 'text': "Hello, I'm a bot. What can I do for you?"}

In [ ]:
no_memory_chain({"question": "I have just introduced myself. What is my name?"})

{'question': 'I have just introduced myself. What is my name?',
 'text': "I'm a chatbot."}

#### With memory - LLMChain

In [ ]:
# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)
# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="chat_history")
with_memory_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [ ]:
with_memory_chain({"question": "ay yo!"})

{'question': 'ay yo!', 'chat_history': '', 'text': 'How can I help you?'}

In [ ]:
memory.save_context({"input": "how yo doin' My name is Junior. Nice to meet you."}, {"output": "Nice to meet you, Junior!. I am fine. Thank you!"})

In [ ]:
with_memory_chain({"question": "what is my name again?"})

{'question': 'what is my name again?',
 'chat_history': "Human: ay yo!\nAI: How can I help you?\nHuman: how yo doin' My name is Junior. Nice to meet you.\nAI: Nice to meet you, Junior!. I am fine. Thank you!",
 'text': 'AI: Your name is Junior.'}

#### With memory - ConversationChain

In [ ]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm,
    verbose=False,
)

In [ ]:
conversation.predict(input="how yo doin' My name is Junior. Nice to meet you.")

"I'm fine. What's your name?"

In [ ]:
conversation.predict(input="what is my name again?")

'Junior.'

### Bonus: Different types of Memory

More info: https://python.langchain.com/docs/modules/memory/types/

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationSummaryMemory(llm=llm)
)

In [ ]:
conversation.predict(input="how yo doin' My name is Junior. Nice to meet you.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: how yo doin' My name is Junior. Nice to meet you.
AI:

> Finished chain.


" Hi Junior, nice to meet you too! I'm doing great, thank you for asking. How about you?"

In [ ]:
conversation.predict(input="I like the color red. My favorite subject is Math.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Junior and the AI responds by introducing itself and asking how Junior is doing.
Human: I like the color red. My favorite subject is Math.
AI:

> Finished chain.


" Hi Junior, I'm AI. Nice to meet you. How are you doing? I'm curious, why do you like the color red? Math is an interesting subject, what do you like about it?"

In [ ]:
conversation.predict(input="What color should i wear?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Junior and the AI responds by introducing itself and asking how Junior is doing. Junior expresses that they like the color red and their favorite subject is Math, to which the AI responds by asking why Junior likes the color red and what they like about Math.
Human: What color should i wear?
AI:

> Finished chain.


" I'm not sure what color you should wear. What colors do you like?"

## 4. Retriever (RAG)

More info: https://python.langchain.com/docs/modules/data_connection/

In [ ]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [ ]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

#### Hugging Face Embeddings

In [ ]:
%pip install sentence_transformers faiss-cpu

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


#### OpenAI Embeddings

In [ ]:
%pip install chromadb tiktoken

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_texts(texts, embeddings)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  

We’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.


#### Retrieval

In [ ]:
retriever = db.as_retriever()

In [ ]:
template = """Answer the question based only on the following context:

{context}

Question: {question}

Let's think step by step.

Answer:
"""

prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm

)

In [ ]:
answer = chain.invoke("Who is Ketanji Brown Jackson?")
print(answer)

One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. We can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling. We’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.


## 5. Evaluation

More info: https://python.langchain.com/docs/guides/evaluation/


### Evaluator

In [ ]:
from langchain.evaluation import load_evaluator
from langchain.evaluation import EvaluatorType

evaluator = load_evaluator(EvaluatorType.CRITERIA, criteria="conciseness")

eval_result = evaluator.evaluate_strings(
    prediction="What's 2+2? That's an elementary question. The answer you're looking for is that two and two is four.",
    input="What's 2+2?",
)

print(f'Evaluation value (Y/N): {eval_result["value"]}')
print(f'Evaluation score: {eval_result["score"]}')
print(f'Evaluation reasoning: {eval_result["reasoning"]}')

Evaluation value (Y/N): N
Evaluation score: 0
Evaluation reasoning: The criterion is conciseness, which means the submission should be brief and to the point. 

Looking at the submission, the answer to the question "What's 2+2?" is given as "The answer you're looking for is that two and two is four." However, before providing the answer, the respondent adds an unnecessary comment: "That's an elementary question." This comment does not contribute to answering the question and thus makes the response less concise.

Therefore, the submission does not meet the criterion of conciseness.

N


In [ ]:
eval_result = evaluator.evaluate_strings(
    prediction="four.",
    input="What's 2+2?",
)

print(f'Evaluation value (Y/N): {eval_result["value"]}')
print(f'Evaluation score: {eval_result["score"]}')
print(f'Evaluation reasoning: {eval_result["reasoning"]}')

Evaluation value (Y/N): Y
Evaluation score: 1
Evaluation reasoning: The criterion is conciseness, which means the submission should be brief and to the point. 

Looking at the submission, the answer to the question "What's 2+2?" is given as "four." This is a direct and succinct response to the question. 

The submission does not include any unnecessary information or details, making it concise. 

Therefore, the submission meets the criterion of conciseness. 

Y


#### Different type of evaluator -- Labeled Criteria

In [ ]:
evaluator = load_evaluator("labeled_criteria", criteria="correctness")

# We can even override the model's learned knowledge using ground truth labels
eval_result = evaluator.evaluate_strings(
    input="What is the capital of the US?",
    prediction="Bangkok",
    reference="The capital of the US is Washington D.C.",
)

print(f'Evaluation value (Y/N): {eval_result["value"]}')
print(f'Evaluation score: {eval_result["score"]}')
print(f'Evaluation reasoning: {eval_result["reasoning"]}')

Evaluation value (Y/N): N
Evaluation score: 0
Evaluation reasoning: The criterion for this task is the correctness of the submitted answer. The input asks for the capital of the US. The submitted answer is "Bangkok", which is incorrect as the capital of the US is Washington D.C., as stated in the reference. Therefore, the submission does not meet the criterion of correctness.

N


In [ ]:
evaluator = load_evaluator("labeled_criteria", criteria="correctness")

# We can even override the model's learned knowledge using ground truth labels
eval_result = evaluator.evaluate_strings(
    input="What is the capital of the US?",
    prediction="Washington D.C.",
    reference="The capital of the US is Washington D.C.",
)

print(f'Evaluation value (Y/N): {eval_result["value"]}')
print(f'Evaluation score: {eval_result["score"]}')
print(f'Evaluation reasoning: {eval_result["reasoning"]}')

Evaluation value (Y/N): Y
Evaluation score: 1
Evaluation reasoning: The criterion for this task is the correctness of the submitted answer. This involves checking if the submission is accurate, factual, and directly answers the given input.

The input asks for the capital of the US. The submitted answer is Washington D.C.

Comparing this with the reference answer, which is also Washington D.C., it is clear that the submitted answer is correct. It is factual and accurate, as Washington D.C. is indeed the capital of the US.

Therefore, the submission meets the criterion.

Y


### Bonus: Generating test datasets & QAEvaluation Chain

In [9]:
%pip install langchain[docarray]

In [11]:
import requests

url = "https://raw.githubusercontent.com/Ryota-Kawamura/LangChain-for-LLM-Application-Development/main/OutdoorClothingCatalog_1000.csv"
res = requests.get(url)
with open("OutdoorClothingCatalog_1000.csv", "w") as f:
  f.write(res.text)

In [24]:
openai_llm = OpenAI(temperature=0)
hf_llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [25]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA

file = "./OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=file)
data = loader.load()

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [27]:
data[10]

Document(page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.", metadata={'source': './OutdoorClothingCatalog_1000.csv', 'row': 10})

In [28]:
data[11]

Document(page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.', metadata={'source': './OutdoorClothingCatalog_1000.csv', 'row': 11})

In [29]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [30]:
from langchain.evaluation.qa import QAGenerateChain

example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [33]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [20]:
new_examples[0]

{'qa_pairs': {'query': "What is the weight of each pair of Women's Campside Oxfords?",
  'answer': "The weight of each pair of Women's Campside Oxfords is approximately 1 lb. 1 oz."}}

In [21]:
data[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': './OutdoorClothingCatalog_1000.csv', 'row': 0})

In [35]:
import langchain
langchain.debug = True

In [36]:
qa = RetrievalQA.from_chain_type(
    llm=hf_llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [37]:
qa.run(examples[0]["query"])

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded h

'yes'

In [45]:
# Turn off the debug mode
langchain.debug = False

In [40]:
new_examples

[{'qa_pairs': {'query': "What is the weight of the Women's Campside Oxfords per pair?",
   'answer': "The weight of the Women's Campside Oxfords per pair is approximately 1 lb. 1 oz."}},
 {'qa_pairs': {'query': 'What are the dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave?',
   'answer': 'The dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave are 18" x 28".'}},
 {'qa_pairs': {'query': "What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?",
   'answer': "The features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece include bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom, and the ability to be machine washed and line dried."}},
 {'qa_pairs': {'query': 'What is the fabric composition of the Refresh Swimwear, V-Neck Tankini Contrasts?',
   'answ

In [43]:
test_data = []
for example in new_examples:
  test_data.append(example['qa_pairs'])

test_data

[{'query': "What is the weight of the Women's Campside Oxfords per pair?",
  'answer': "The weight of the Women's Campside Oxfords per pair is approximately 1 lb. 1 oz."},
 {'query': 'What are the dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave are 18" x 28".'},
 {'query': "What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?",
  'answer': "The features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece include bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom, and the ability to be machine washed and line dried."},
 {'query': 'What is the fabric composition of the Refresh Swimwear, V-Neck Tankini Contrasts?',
  'answer': 'The Refresh Swimwear, V-Neck Tankini Contrasts is mad

In [51]:
predictions = qa.apply(test_data)
predictions



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


[{'query': "What is the weight of the Women's Campside Oxfords per pair?",
  'answer': "The weight of the Women's Campside Oxfords per pair is approximately 1 lb. 1 oz.",
  'result': '1 lb.1 oz.'},
 {'query': 'What are the dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave are 18" x 28".',
  'result': '18" x 28".'},
 {'query': "What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?",
  'answer': "The features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece include bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom, and the ability to be machine washed and line dried.",
  'result': "Our four-way-stretch and chlorine-resistant fabric keeps its shape and resists snags. The UPF 50+ 

In [46]:
from langchain.evaluation.qa import QAEvalChain

eval_chain = QAEvalChain.from_llm(openai_llm)

In [47]:
graded_outputs = eval_chain.evaluate(test_data, predictions)

In [49]:
graded_outputs[0]

{'results': ' CORRECT'}

In [53]:
for i, eg in enumerate(new_examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print('Verdict: ' + graded_outputs[i]['results'])
    print()

Example 0:
Question: What is the weight of the Women's Campside Oxfords per pair?
Real Answer: The weight of the Women's Campside Oxfords per pair is approximately 1 lb. 1 oz.
Predicted Answer: 1 lb.1 oz.
Verdict:  CORRECT

Example 1:
Question: What are the dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave?
Real Answer: The dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave are 18" x 28".
Predicted Answer: 18" x 28".
Verdict:  CORRECT

Example 2:
Question: What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?
Real Answer: The features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece include bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom, and the ability to be machine washed and line dried.
Predicted Answer: Our four-way-stretch and chlorine-re